In [1]:
import numpy as np
import pandas as pd
from numerical_methods.eigen_system import power_method, inverse_power_method
import time

In [2]:
def rand_mat(n=50):
    B = np.random.normal(size=(n, n))
    A = np.eye(n) / 2 + B + B.T
    return A


def vec_err(A, x, val, val_true):
    return np.linalg.norm(A @ x - val * x, ord=np.inf) / val_true

In [3]:
pow_time = []
pow_val_err = []
pow_vec_err = []
inv_time = []
inv_val_err = []
inv_vec_err = []

for _ in range(25):
    A = rand_mat(n=50)
    x0 = np.ones(shape=(50,1))
    
    val_true, vec_true = np.linalg.eig(A)
    max_idx = np.argmax(np.abs(val_true))
    min_idx = np.argmin(np.abs(val_true))
    max_val_true = val_true[max_idx]
    max_vec_true = vec_true[max_idx]
    min_val_true = val_true[min_idx]
    min_vec_true = vec_true[min_idx]
    
    pow_start = time.time()
    val_pow, vec_pow = power_method(A, x0).unwrap()
    pow_time.append(time.time() - pow_start)
    pow_val_err.append(np.abs(val_pow - max_val_true))
    pow_vec_err.append(vec_err(A, vec_pow, val_pow, max_vec_true))
    
    inv_start = time.time()
    val_inv, vec_inv = inverse_power_method(A, x0, q=0).unwrap()
    inv_time.append(time.time() - inv_start)
    inv_val_err.append(np.abs(val_inv - min_val_true))
    inv_vec_err.append(vec_err(A, vec_inv, val_inv, min_vec_true))

In [4]:
df = pd.DataFrame({
    "Smallest Eig": [np.mean(xs) for xs in [inv_time, inv_val_err, inv_vec_err]],
    "Largest Eig": [np.mean(xs) for xs in [pow_time, pow_val_err, pow_vec_err]]   
}).T
df = df.set_axis(["lambda time", "lambda error", "Vector Error"], axis=1)
df

,lambda time,lambda error,Vector Error
Smallest Eig,0.009330,0.000003,0.000067
Largest Eig,0.009005,0.000636,0.036263
